In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [3]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data=data.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [6]:
label_gender=LabelEncoder()
data['Gender']=label_gender.fit_transform(data['Gender'])

In [11]:
label_geo=OneHotEncoder(handle_unknown='ignore')
label_encode_geo=label_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(label_encode_geo,columns=label_geo.get_feature_names_out(['Geography']))

In [14]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [15]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [16]:
X=data.drop("EstimatedSalary",axis=1)
y=data['EstimatedSalary']

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=21)

In [18]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [19]:
with open('label_gender.pkl','wb') as file:
    pickle.dump(label_gender,file)
with open('label_geo.pkl','wb') as file:
    pickle.dump(label_geo,file)
with open('scaler1.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN regression problem statement

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
model1=Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])
model1.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
## setup the tensorboard
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir="logs1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [28]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [29]:
# Training the model
history=model1.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
    )
                   

Epoch 1/100


250/250 [==============================] - 2s 4ms/step - loss: 99900.9766 - mae: 99900.9766 - val_loss: 100465.7188 - val_mae: 100465.7188
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99199.3984 - mae: 99199.3984 - val_loss: 99023.6172 - val_mae: 99023.6172
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96664.2656 - mae: 96664.2656 - val_loss: 95291.7109 - val_mae: 95291.7109
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 91624.4375 - mae: 91624.4375 - val_loss: 89072.7578 - val_mae: 89072.7578
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 84251.6484 - mae: 84251.6484 - val_loss: 80847.3906 - val_mae: 80847.3906
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 75428.3750 - mae: 75428.3750 - val_loss: 72038.6562 - val_mae: 72038.6562
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 66783.5859 - mae: 66783.585

In [30]:
%load_ext tensorboard

In [32]:
%tensorboard --logdir logs1/fit

In [33]:
## evaluate the model
test_loss,test_mae=model1.evaluate(X_test,y_test)

63/63 [==============================] - 0s 2ms/step - loss: 49839.7188 - mae: 49839.7188


In [34]:
model1.save('regressionModel.h5')

c:\Users\akhil.goud.bathini\Desktop\GenAI\Projects\AnnCls\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
